In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit_aer import Aer
from qiskit import execute
import random
import numpy as np
import sys
sys.path.append("../")
from qmg.utils import MoleculeQuantumStateGenerator, CircuitBuilder
from typing import List, Union
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# from cusvaer.backends import StatevectorSimulator



ImportError: cannot import name 'QuantumCircuit' from 'qiskit.circuit' (unknown location)

In [4]:
num_heavy_atom = 4
num_sample = 2**15
assert num_heavy_atom >= 2
num_qubits = 4 + (num_heavy_atom-1) * 2
num_clbits = num_heavy_atom * (num_heavy_atom + 1)
num_weights = int(8 + (num_heavy_atom - 2)*(num_heavy_atom + 3) * 3 / 2)
print(num_weights)

data_generator = MoleculeQuantumStateGenerator(num_heavy_atom)
qubits = QuantumRegister(num_qubits)
clbits = ClassicalRegister(num_clbits)
qc = QuantumCircuit(qubits, clbits)
random.seed(1)
weight_vector = [random.random() for _ in range(num_weights)]

def get_classical_register_by_name(qc: QuantumCircuit, name: str):
    for cr in qc.cregs:
        if cr.name == name:
            return cr
    else:
        raise ValueError(f"The {name} is not found in classical registers.")

def softmax_temperature(weight_vector, temperature=0.2):
    weight_vector /= temperature
    exps = np.exp(weight_vector)
    return exps / np.sum(exps)

def controlled_ry(control:int, target:int, digit:float):
    qc.cry(np.pi*digit, control, target)

def reset_qubit(qubit, cbit):
    with qc.if_test((clbits[cbit], 1)):
        qc.x(qubit)

def build_two_atom_circuit(qc, weight_vector, qubits, clbits):
    qc.ry(np.pi * weight_vector[0], 0)
    qc.x(1)
    qc.ry(np.pi * weight_vector[2], 2)
    qc.ry(np.pi * weight_vector[4], 3)
    qc.cx(0, 1)
    controlled_ry(1, 2, weight_vector[3])
    qc.cx(2, 3)
    controlled_ry(0, 1, weight_vector[1])
    qc.cx(1, 2)
    controlled_ry(2, 3, weight_vector[5])

    # measure atom 1 state:
    qc.measure(qubits[0:2], clbits[0:2])
    # measure atom 2 state and save:
    qc.measure(qubits[2:4], clbits[2:4])

    # Add a new atom existence ClassicalRegister:
    atom_existence_CR = ClassicalRegister(bits=clbits[2:4], name="atom_2_existence")
    qc.add_register(atom_existence_CR)
    with qc.if_test((atom_existence_CR, 0)) as else_:
        pass
    with else_:
        qc.ry(np.pi * weight_vector[6], 4)
        qc.x(5)
        qc.cx(4,5)
        controlled_ry(4, 5, weight_vector[7])

    qc.measure(qubits[4:6], clbits[4:6])

def reset_previous_atom_bond_circuit(qc, heavy_idx):
    reset_qubits_index = list(range(2, 2*heavy_idx))
    start_clbit = (heavy_idx - 2)**2 + (heavy_idx - 2)
    reset_clbits_index = list(range(start_clbit, start_clbit+(heavy_idx - 1)*2))
    for qubit_index, clbit_index in zip(reset_qubits_index, reset_clbits_index):
        with qc.if_test((clbits[clbit_index], 1)):
            qc.x(qubit_index)

def build_atom_type_circuit(qc, heavy_idx: int, weight_vector: Union[List[float], np.ndarray]):
    assert len(weight_vector) == 3
    qubit_1_index = 2
    qubit_2_index = 3
    clbit_1_index = (heavy_idx - 1)**2 + (heavy_idx - 1)
    clbit_2_index = clbit_1_index + 1

    register_name = f"atom_{heavy_idx-1}_existence"
    atom_existence_CR = get_classical_register_by_name(qc, register_name)
    with qc.if_test((atom_existence_CR, 0)) as else_:
        pass
    with else_:
        qc.ry(np.pi * weight_vector[0], qubit_1_index)
        qc.ry(np.pi * weight_vector[1], qubit_2_index)
        qc.cx(qubit_1_index, qubit_2_index)
        controlled_ry(4, 5, weight_vector[2])
    qc.measure(qubits[[qubit_1_index,qubit_2_index]], clbits[[clbit_1_index,clbit_2_index]])
    # create new atom existence register
    atom_existence_CR = ClassicalRegister(bits=clbits[[clbit_1_index,clbit_2_index]], name=f"atom_{heavy_idx}_existence")
    qc.add_register(atom_existence_CR)

def build_bond_type_circuit(qc, heavy_idx: int, fixed_weight_vector: Union[List[float], np.ndarray],
                                flexible_weight_vector: Union[List[float], np.ndarray], remove_bond_disconnection=True):
    assert len(fixed_weight_vector) == heavy_idx-1
    assert len(flexible_weight_vector) == 2*(heavy_idx-1)
    qubit_start_index = 4
    qubit_end_index = qubit_start_index + 2*(heavy_idx - 1)
    clbit_start_index = (heavy_idx)**2 - heavy_idx + 2
    clbit_end_index = clbit_start_index + 2*(heavy_idx - 1)

    register_name = f"atom_{heavy_idx}_existence"
    atom_existence_CR = get_classical_register_by_name(qc, register_name)
    with qc.if_test((atom_existence_CR, 0)) as else_:
        pass
    with else_:
        for i in range(heavy_idx-1):
            qc.ry(np.pi * fixed_weight_vector[i], qubit_start_index+2*i+1)
            controlled_ry(qubit_start_index+2*i+1, qubit_start_index+2*i, flexible_weight_vector[2*i]) # < 0.5
            controlled_ry(qubit_start_index+2*i, qubit_start_index+2*i+1, flexible_weight_vector[2*i+1]) # > 0.5
        qc.measure(qubits[qubit_start_index:qubit_end_index], clbits[clbit_start_index:clbit_end_index])
        if remove_bond_disconnection:
            bond_disconnection_CR = ClassicalRegister(bits=clbits[clbit_start_index:clbit_end_index], name=f"bond_{heavy_idx}_connection")
            qc.add_register(bond_disconnection_CR)
            with qc.if_test((bond_disconnection_CR, 0)):
                qc.x(qubit_end_index-1)
                qc.measure(qubits[qubit_end_index-1], clbits[clbit_end_index-1])
    return

build_two_atom_circuit(qc, weight_vector, qubits, clbits)
for heavy_idx in range(3, num_heavy_atom+1):
    atom_type_weights = [random.random() for _ in range(3)]
    bond_type_fixed_weight_vector = np.array([random.random() for _ in range((heavy_idx-1))])
    bond_type_fixed_weight_vector = softmax_temperature(bond_type_fixed_weight_vector, temperature=0.2)
    bond_type_flexible_weight_vector = np.array([random.random()*0.5 for _ in range((heavy_idx-1)*2)])
    bond_type_flexible_weight_vector += np.array([0, 0.5]*(heavy_idx-1))
    print(bond_type_fixed_weight_vector)
    print(bond_type_flexible_weight_vector)

    reset_previous_atom_bond_circuit(qc, heavy_idx)
    build_atom_type_circuit(qc, heavy_idx, atom_type_weights)
    build_bond_type_circuit(qc, heavy_idx, bond_type_fixed_weight_vector,
                                bond_type_flexible_weight_vector, remove_bond_disconnection=True)
# simulator = Aer.get_backend('aer_simulator')
# results = execute(qc, backend=simulator, shots=num_sample).result()
# counts = results.get_counts(qc)
# print(counts)

simulator = Aer.get_backend('aer_simulator_statevector')
circuit = transpile(qc, simulator)
job = simulator.run(circuit)
result = job.result()
print(result.get_counts())

# qc.draw("mpl", style="mpl")
qc.draw("text")

29
[0.2307445 0.7692555]
[0.14489081 0.51074485 0.41878899 0.77822716]
[0.91201822 0.02265492 0.06532686]
[0.3607422  0.85559588 0.46822029 0.7110535  0.41501785 0.83515278]
{'100000 01 1000 10 10 1010': 1, '000010 11 1000 10 10 1010': 1, '100000 11 1000 11 11 1010': 1, '000010 10 0110 10 11 1010': 1, '000010 01 1010 11 10 1010': 1, '000000 00 1000 11 11 1111': 1, '000010 11 0100 10 11 1010': 1, '000010 10 1100 01 11 1010': 1, '000000 00 1100 11 10 1010': 1, '100000 01 1000 11 10 1010': 1, '000000 00 0100 11 10 1111': 1, '000000 00 0000 00 11 1110': 1, '000010 11 0100 10 01 1111': 1, '000010 11 1000 01 01 1010': 2, '000010 01 1000 11 10 1010': 2, '000000 00 0000 00 11 1111': 1, '000010 11 1100 11 11 1010': 2, '000010 11 1001 11 10 1010': 1, '000001 11 0100 11 11 1010': 3, '000001 11 1100 11 10 1010': 1, '000001 11 1000 11 01 1010': 2, '000010 11 0010 10 10 1010': 1, '000010 01 1100 11 11 1010': 1, '000000 00 1000 11 11 1011': 1, '000010 01 0100 11 11 1010': 1, '000000 00 0110 01 11 101

┌─────────────┐                                       ┌─┐              »
q21_0: ┤ Ry(0.42212) ├──■───────────────────────■────────────┤M├──────────────»
       └────┬───┬────┘┌─┴─┐               ┌─────┴──────┐     └╥┘              »
q21_1: ─────┤ X ├─────┤ X ├───────■───────┤ Ry(2.6623) ├──■───╫───────────────»
        ┌───┴───┴────┐└───┘┌──────┴──────┐└────────────┘┌─┴─┐ ║               »
q21_2: ─┤ Ry(2.3995) ├─────┤ Ry(0.80132) ├──────■───────┤ X ├─╫───────■───────»
        ├────────────┤     └─────────────┘    ┌─┴─┐     └───┘ ║ ┌─────┴──────┐»
q21_3: ─┤ Ry(1.5565) ├────────────────────────┤ X ├───────────╫─┤ Ry(1.4121) ├»
        └────────────┘                        └───┘           ║ └────────────┘»
q21_4: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q21_5: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q21_6: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q21_7: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q21_8: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
q21_9: ───────────────────────────────────────────────────────╫───────────────»
                                                              ║               »
c2: 20/═══════════════════════════════════════════════════════╩═══════════════»
                                                              0               »
«                                                                            »
«q21_0: ─────────────────────────────────────────────────────────────────────»
«       ┌─┐                                                                  »
«q21_1: ┤M├──────────────────────────────────────────────────────────────────»
«       └╥┘┌─┐                                                               »
«q21_2: ─╫─┤M├───────────────────────────────────────────────────────────────»
«        ║ └╥┘┌─┐                                                            »
«q21_3: ─╫──╫─┤M├────────────────────────────────────────────────────────────»
«        ║  ║ └╥┘┌────── ┌──────── ┌───────────┐                    ───────┐ »
«q21_4: ─╫──╫──╫─┤       ┤         ┤ Ry(2.047) ├──■────────■───────        ├─»
«        ║  ║  ║ │ If-0  │ Else-0  └───┬───┬───┘┌─┴─┐┌─────┴──────┐  End-0 │ »
«q21_5: ─╫──╫──╫─┤       ┤         ────┤ X ├────┤ X ├┤ Ry(2.4778) ├        ├─»
«        ║  ║  ║ └──╥─── └────────     └───┘    └───┘└────────────┘ ───────┘ »
«q21_6: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
«q21_7: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
«q21_8: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║    ║                                                        »
«q21_9: ─╫──╫──╫────╫────────────────────────────────────────────────────────»
«        ║  ║  ║ ┌──╨──┐                                                     »
«c2: 20/═╩══╩══╩═╡ 0x0 ╞═════════════════════════════════════════════════════»
«        1  2  3 └─────┘                                                     »
«                                                                   »
«q21_0: ────────────────────────────────────────────────────────────»
«                                                                   »
«q21_1: ────────────────────────────────────────────────────────────»
«               ┌──────   ┌───┐ ───────┐                            »


In [5]:
smiles_dict = {}
num_valid_molecule = 0
for key, value in counts.items():
    key = "".join(key.split())
    smiles = data_generator.QuantumStateToSmiles(data_generator.post_process_quantum_state(key))
    smiles_dict[smiles] = smiles_dict.get(smiles, 0) + value
    if smiles:
        num_valid_molecule += value
validity = num_valid_molecule / num_sample
diversity = (len(smiles_dict.keys()) - 1) / num_sample
print(smiles_dict)
print(validity, diversity)



{None: 3475, 'N=NN=N': 1, 'NN1C=N1': 1, 'C1=NNC1': 3, 'C1C2OC12': 1, 'N=CNO': 1, 'N#CN=N': 1, 'N=NO': 2, 'CC=NN': 5, 'C1NN1': 1, 'C#CCO': 3, 'CCN=N': 2, 'O=CC=O': 2, 'CC=CO': 5, 'C=C(C)N': 1, 'C=N': 22, 'NNOO': 1, 'CNCN': 3, 'OC1=CN1': 2, 'C=CCC': 3, 'CC(=N)N': 2, 'NN=NO': 1, 'C#CCC': 2, 'N=C=NN': 2, 'C=CNC': 1, 'NNCO': 2, 'CC#CO': 18, 'C#CCN': 8, 'CN=O': 1, 'NC1CC1': 5, '[N-]=[N+]=N': 2, 'CCC=O': 8, 'CCNN': 15, 'N#CCN': 32, 'OC#CO': 12, 'c1ooc#1': 1, 'CC=O': 13, 'CC1C=N1': 1, 'CC=C=N': 8, 'CC(C)N': 6, 'C1#CNC1': 1, 'C1=CN1': 8, 'CCCO': 6, 'NCN': 7, 'C#CN': 744, 'OC1C=N1': 2, 'C=CC': 32, 'C1#CC1': 10, 'N=CN=N': 3, 'N=CCO': 59, 'C1CC1': 4, 'CN': 80, 'C#CC': 97, 'CC=NC': 1, 'NCCN': 485, 'C=CC=N': 19, 'NC1=CC1': 4, 'N=O': 5, 'N=C=CN': 98, 'NCCO': 99, 'N=C=C=O': 8, 'O=CCO': 5, 'C=CCN': 39, 'O': 2, 'NO': 6, 'CC=CN': 18, 'CC(N)O': 1, 'C=CON': 2, 'NC#CO': 258, 'C=CO': 10, 'C1CNC1': 1, 'CCN': 284, 'N=C1CN1': 16, 'C=CC#N': 2, 'C1CO1': 8, 'N=NN': 1, 'c1[nH][nH]c#1': 8, 'CNN': 2, 'C#CNN': 30, 'C=

In [19]:
import numpy as np

one_one = np.array([[1, 0], [0, 0]])
zero_zero = np.array([[0, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

A = np.kron(one_one, X)
B = np.kron(zero_zero, H)
C = A+B
print(C)
# print(C.conjugate().transpose())
# print(np.matmul(C, C.conjugate().transpose()))
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from qiskit.quantum_info.synthesis import two_qubit_cnot_decompose
from qiskit.quantum_info import random_unitary

# 創建一個隨機的 4x4 單位矩陣
unitary = C

# 使用 two_qubit_cnot_decompose 將 4x4 矩陣分解為兩個 qubits 的量子門
decomposed_circuit = two_qubit_cnot_decompose(Operator(unitary))

# 打印分解後的量子電路
# print(decomposed_circuit)
decomposed_circuit.draw(style="mpl")

[[ 0.          1.          0.          0.        ]
 [ 1.          0.          0.          0.        ]
 [ 0.          0.          0.70710678  0.70710678]
 [ 0.          0.          0.70710678 -0.70710678]]


global phase: 6.1208
       ┌───────────────┐        ┌───────────────────────┐       »
q6_0: ─┤ U(π/2,0,-π/2) ├────■───┤ U(π,0.84253,-0.72827) ├────■──»
      ┌┴───────────────┴─┐┌─┴─┐┌┴───────────────────────┴─┐┌─┴─┐»
q6_1: ┤ U(0.40375,π/2,0) ├┤ X ├┤ U(1.4648,-1.2877,2.2836) ├┤ X ├»
      └──────────────────┘└───┘└──────────────────────────┘└───┘»
«      ┌──────────────────┐
«q6_0: ┤ U(π/2,-π/2,3π/4) ├
«      ├──────────────────┤
«q6_1: ┤ U(1.9745,-π,π/2) ├
«      └──────────────────┘

In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

def create_ghz_circuit(n_qubits):
    circuit = QuantumCircuit(n_qubits)
    circuit.h(0)
    for qubit in range(n_qubits - 1):
        circuit.cx(qubit, qubit + 1)
    return circuit

simulator = Aer.get_backend('aer_simulator_statevector')
circuit = create_ghz_circuit(n_qubits=20)
circuit.measure_all()
circuit = transpile(circuit, simulator)
job = simulator.run(circuit)
result = job.result()

print(result.get_counts())
print(f'backend: {result.backend_name}')

ModuleNotFoundError: No module named 'qiskit'